In [1]:
import pandas as pd
import numpy as np

from cart import *

In [2]:
pd.options.display.max_columns = None
dataframe = pd.read_table('lpmc.dat')
print(dataframe.shape)
dataframe.head()

(81086, 32)


,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,travel_year,travel_month,travel_date,day_of_week,start_time,age,female,driving_license,car_ownership,distance,dur_walking,dur_cycling,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent
0,0,0,0,0,4,3,1,1,1.0,1,2012,4,1,7,10.000000,56,1,1,2,778,0.218056,0.092222,0.134444,0.0,0.016667,0.0,0,0.052222,1.5,0.14,0.0,0.111702
1,1,0,0,1,4,3,1,1,1.0,1,2012,4,1,7,13.000000,56,1,1,2,778,0.188889,0.061944,0.109444,0.0,0.055556,0.0,0,0.059444,1.5,0.15,0.0,0.112150
2,2,0,0,2,4,3,1,1,1.0,1,2012,4,1,7,17.000000,56,1,1,2,4562,1.037778,0.355833,0.203056,0.0,0.210278,0.0,0,0.236667,1.5,0.79,0.0,0.203052
3,3,0,0,3,4,3,1,1,1.0,1,2012,4,1,7,17.350000,56,1,1,2,4562,1.039444,0.384444,0.205556,0.0,0.258611,0.0,0,0.233333,1.5,0.78,0.0,0.160714
4,4,0,1,2,4,3,1,4,1.0,1,2012,4,1,7,19.583333,54,0,1,2,4562,1.037778,0.355833,0.203056,0.0,0.189444,0.0,0,0.229167,1.5,0.78,0.0,0.130909


In [3]:
#binning age

groupDict={'0-18':0,'19-30':0,'31-65':0, '66+': 0}

for i in dataframe['age']:
    if i>=0 and i<=10:
        groupDict['0-18']+=1
    elif i>=19 and i<=30:
        groupDict['19-30']+=1
    elif i>=31 and i<=65:
        groupDict['31-65']+=1
    else:
        groupDict['66+']+=1

dataframe.loc[dataframe['age']<=18, 'age_group'] = '0-18'
dataframe.loc[dataframe['age'].between(19,30), 'age_group'] = '19-30'
dataframe.loc[dataframe['age'].between(31,65), 'age_group'] = '31-65'
dataframe.loc[dataframe['age']>65, 'age_group'] = '66+'
dataframe.drop(['age'], axis= 1, inplace = True)


#binning distance

dataframe.loc[dataframe['distance']< 658.50, 'distance_group'] = 'distance01'
dataframe.loc[dataframe['distance'].between(658.50,861.49), 'distance_group'] = 'distance02'
dataframe.loc[dataframe['distance'].between(861.50,1214.49), 'distance_group'] = 'distance03'
dataframe.loc[dataframe['distance'].between(1214.50,1514.49 ), 'distance_group'] = 'distance04'
dataframe.loc[dataframe['distance'].between(1514.50, 2156.49), 'distance_group'] = 'distance05'
dataframe.loc[dataframe['distance'].between(2156.50, 2759.49), 'distance_group'] = 'distance06'
dataframe.loc[dataframe['distance'].between(2759.50, 4391.49), 'distance_group'] = 'distance07'
dataframe.loc[dataframe['distance'].between(4391.50, 5369.49), 'distance_group'] = 'distance08'
dataframe.loc[dataframe['distance'].between(5369.50, 10087.49), 'distance_group'] = 'distance09'
dataframe.loc[dataframe['distance'].between(10087.50, 14695.49), 'distance_group'] = 'distance10'
dataframe.loc[dataframe['distance']>=14695.50, 'distance_group'] = 'distance11'
dataframe.drop('distance', axis = 1, inplace = True)

#binning start time

dataframe.loc[dataframe['start_time']<8.091666, 'starttime_group'] = 'early_morning'
dataframe.loc[dataframe['start_time'].between(8.091666,10.00833320), 'starttime_group'] = 'morning'
dataframe.loc[dataframe['start_time'].between(10.008333211,18.50833320), 'starttime_group'] = 'afternoon'
dataframe.loc[dataframe['start_time']> 18.50833321, 'starttime_group'] = 'evening'
dataframe.drop('start_time', axis = 1, inplace = True)

#binning cost transit

dataframe.loc[dataframe['cost_transit']< 1.52499998, 'cost_group'] = 'low_cost'
dataframe.loc[dataframe['cost_transit'].between(1.52499998,3.01999997), 'cost_group'] = 'medium_cost'
dataframe.loc[dataframe['cost_transit']> 3.01999998, 'cost_group'] = 'high_cost'
dataframe.drop('cost_transit', axis = 1, inplace = True)

#binning cost_driving_Fuel

dataframe.loc[dataframe['cost_driving_fuel']< 0.165, 'costdrivingfuel_group'] = 'costdrivingfuelgroup1'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.165,0.204), 'costdrivingfuel_group'] = 'costdrivingfuelgroup2'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.205,0.244), 'costdrivingfuel_group'] = 'costdrivingfuelgroup3'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.245, 0.294), 'costdrivingfuel_group'] = 'costdrivingfuelgroup4'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.295, 0.354), 'costdrivingfuel_group'] = 'costdrivingfuelgroup5'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.355, 0.44499998), 'costdrivingfuel_group'] = 'costdrivingfuelgroup6'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.44499999, 0.83499997), 'costdrivingfuel_group'] = 'costdrivingfuelgroup7'
dataframe.loc[dataframe['cost_driving_fuel'].between(0.83499998, 1.17499994), 'costdrivingfuel_group'] = 'costdrivingfuelgroup8'
dataframe.loc[dataframe['cost_driving_fuel'].between(1.17499995, 2.31499993), 'costdrivingfuel_group'] = 'costdrivingfuelgroup9'
dataframe.loc[dataframe['cost_driving_fuel']> 2.31499994, 'costdrivingfuel_group'] = 'costdrivingfuelgroup10'
dataframe.drop('cost_driving_fuel', axis = 1, inplace = True)

#binning driving traddic percent

dataframe.loc[dataframe['driving_traffic_percent']< 0.24422751, 'trafficperc_group'] = '0low_traffic'
dataframe.loc[dataframe['driving_traffic_percent'].between(0.24422751,0.36449698), 'trafficperc_group'] = '1mediumlow_traffic'
dataframe.loc[dataframe['driving_traffic_percent'].between(0.36449699,0.49620750), 'trafficperc_group'] = '2medium_traffic'
dataframe.loc[dataframe['driving_traffic_percent'].between(0.49620751, 0.57161700), 'trafficperc_group'] = '3mediumhigh_traffic'
dataframe.loc[dataframe['driving_traffic_percent']> 0.57161701, 'trafficperc_group'] = '4high_traffic'
dataframe.drop('driving_traffic_percent', axis = 1, inplace = True)

#binning cost_driving_ccharge

dataframe.loc[dataframe['cost_driving_ccharge']< 5.25, 'drivingcharge_group'] = 'low_charge'
dataframe.loc[dataframe['cost_driving_ccharge']> 5.25, 'drivingcharge_group'] = 'high_charge'
dataframe.drop('cost_driving_ccharge', axis = 1, inplace = True)

In [4]:
y = dataframe[['travel_mode', 'travel_year']]
X = dataframe.drop(['travel_mode', 'survey_year'], axis = 1)

print(set(X["travel_year"]))

a = X.loc[X["travel_year"]==2015]
#creating test set 

X_test= X.query('travel_year == 2015') #creates a subset, like this: X.loc[X["travel_year"]==2015]
y_test = y.query('travel_year == 2015')

X_test = X_test.drop(['travel_year'], axis =1).copy()
y_test = y_test.drop(['travel_year'], axis =1).copy()

#create training set 

X_train_1 = X.query('travel_year == 2012')
y_train_1 = y.query('travel_year == 2012')
X_train_2 = X.query('travel_year == 2013')
y_train_2 = y.query('travel_year == 2013')
X_train_3 = X.query('travel_year == 2014')
y_train_3 = y.query('travel_year == 2014')

X_train = pd.concat([X_train_1, X_train_2, X_train_3])
y_train = pd.concat([y_train_1, y_train_2, y_train_3])

X_train = X_train.drop(['travel_year'], axis = 1)
y_train = y_train.drop(['travel_year'], axis = 1)


data_to_shuffle = pd.concat([X_train, y_train], axis = 1)
data = data_to_shuffle.sample(frac=1).reset_index(drop=True) #sampling returning all, and resetting index , no seed?
y_train = data['travel_mode']
X_train = data.drop(['travel_mode'], axis = 1)

#feature selection

to_drop = ['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_date', 'dur_pt_int',
          'dur_pt_rail', 'dur_pt_access', 'dur_pt_bus', 'dur_walking', 'dur_cycling', 'dur_driving']
print(f'Number of columns dropped: {len(to_drop)}')
X_train.drop(to_drop, axis = 1, inplace = True)

X_test.drop(to_drop, axis = 1, inplace = True)

predictors = X_test.columns
y = y_test #is the whole vector
print(predictors)

datatest = pd.concat([y_test, X_test], axis = 1).reset_index()
datatest.drop('index', axis = 1, inplace = True)
datatest.to_csv("data_test.csv", index = False)

datatraining = pd.concat([y_train, X_train], axis = 1).reset_index()
datatraining.drop('index', axis = 1, inplace = True)
datatraining.to_csv("data_training.csv", index = False)


{2012, 2013, 2014, 2015}
Number of columns dropped: 12
Index(['purpose', 'fueltype', 'faretype', 'bus_scale', 'travel_month',
       'day_of_week', 'female', 'driving_license', 'car_ownership',
       'pt_interchanges', 'age_group', 'distance_group', 'starttime_group',
       'cost_group', 'costdrivingfuel_group', 'trafficperc_group',
       'drivingcharge_group'],
      dtype='object')


In [5]:
X_train.reset_index(inplace = True, drop = True)
y_train.reset_index(inplace = True, drop = True)

#X_train=X_train.applymap(str) moves elements into strings
#y_train=y_train.astype(str)

n_features = X_train
n_features_names = list(X_train.columns)
indici = np.arange(0, len(y_train))
y = y_train





In [6]:
n_features['purpose'] = 'purpose' + n_features['purpose'].astype(str)
n_features['fueltype'] = 'fueltype' + n_features['fueltype'].astype(str)
n_features['faretype'] = 'faretype' + n_features['faretype'].astype(str)
n_features['bus_scale'] = 'bus_scale' + n_features['bus_scale'].astype(str)
n_features['travel_month'] = 'travel_month' + n_features['travel_month'].astype(str)
n_features['day_of_week'] = 'day_of_week' + n_features['day_of_week'].astype(str)
n_features['female'] = 'female' + n_features['female'].astype(str)
n_features['driving_license'] = 'driving_license' + n_features['driving_license'].astype(str)
n_features['car_ownership'] = 'car_ownership' + n_features['car_ownership'].astype(str)
n_features['pt_interchanges'] = 'pt_interchanges' + n_features['pt_interchanges'].astype(str)
n_features['age_group'] = 'age_group' + n_features['age_group'].astype(str)


n_features.head()

,purpose,fueltype,faretype,bus_scale,travel_month,day_of_week,female,driving_license,car_ownership,pt_interchanges,age_group,distance_group,starttime_group,cost_group,costdrivingfuel_group,trafficperc_group,drivingcharge_group
0,purpose3,fueltype2,faretype5,bus_scale0.0,travel_month10,day_of_week5,female1,driving_license0,car_ownership2,pt_interchanges0,age_group0-18,distance04,afternoon,low_cost,costdrivingfuelgroup2,0low_traffic,low_charge
1,purpose3,fueltype6,faretype5,bus_scale0.0,travel_month1,day_of_week6,female0,driving_license0,car_ownership0,pt_interchanges0,age_group66+,distance06,morning,low_cost,costdrivingfuelgroup4,0low_traffic,low_charge
2,purpose4,fueltype6,faretype5,bus_scale0.0,travel_month3,day_of_week3,female1,driving_license0,car_ownership0,pt_interchanges1,age_group31-65,distance08,afternoon,low_cost,costdrivingfuelgroup7,4high_traffic,low_charge
3,purpose5,fueltype1,faretype5,bus_scale0.0,travel_month6,day_of_week3,female1,driving_license0,car_ownership1,pt_interchanges1,age_group0-18,distance07,afternoon,low_cost,costdrivingfuelgroup7,0low_traffic,low_charge
4,purpose1,fueltype1,faretype5,bus_scale0.0,travel_month11,day_of_week5,female1,driving_license1,car_ownership1,pt_interchanges1,age_group19-30,distance10,early_morning,low_cost,costdrivingfuelgroup9,2medium_traffic,low_charge


In [7]:
features = []
features_names = []


In [8]:
from collections import Counter
import itertools


In [9]:
combinazioni = []
distinct_values= []
distinct_values.append(list(set(y)))
distinct_values = list(itertools.chain(*distinct_values)) #flattens, removed nesting
for i in range(1,len(distinct_values)):
    combinazioni.append(list(itertools.combinations(distinct_values, i)))
combinazioni=combinazioni[1:]
combinazioni = list(itertools.chain(*combinazioni))
combinazioni = combinazioni +  distinct_values
combinazioni

[(1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4),
 (1, 2, 3),
 (1, 2, 4),
 (1, 3, 4),
 (2, 3, 4),
 1,
 2,
 3,
 4]

In [10]:
c1, c2 = [], []
for i in combinazioni:
        if isinstance(i, int): 
            if [i] not in c2:
                c1.append([i])
            if list(set(y)-{i}) not in c1:
                c2.append(list(set(y)-{i}))
        else:
            if list(i) not in c2:
                c1.append(list(i))
            if list(set(y) - set(tuple(i))) not in c1:
                c2.append(list(set(y) - set(tuple(i))))
            
    
               
c1,c2, len(c1)

([[1, 2], [1, 3], [1, 4], [1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]],
 [[3, 4], [2, 4], [2, 3], [4], [3], [2], [1]],
 7)

Twoing 

In [11]:
y.head(), features, features_names, n_features.head(), n_features_names

(0    4
 1    3
 2    3
 3    4
 4    3
 Name: travel_mode, dtype: int64,
 [],
 [],
     purpose   fueltype   faretype     bus_scale    travel_month   day_of_week  \
 0  purpose3  fueltype2  faretype5  bus_scale0.0  travel_month10  day_of_week5   
 1  purpose3  fueltype6  faretype5  bus_scale0.0   travel_month1  day_of_week6   
 2  purpose4  fueltype6  faretype5  bus_scale0.0   travel_month3  day_of_week3   
 3  purpose5  fueltype1  faretype5  bus_scale0.0   travel_month6  day_of_week3   
 4  purpose1  fueltype1  faretype5  bus_scale0.0  travel_month11  day_of_week5   
 
     female   driving_license   car_ownership   pt_interchanges  \
 0  female1  driving_license0  car_ownership2  pt_interchanges0   
 1  female0  driving_license0  car_ownership0  pt_interchanges0   
 2  female1  driving_license0  car_ownership0  pt_interchanges1   
 3  female1  driving_license0  car_ownership1  pt_interchanges1   
 4  female1  driving_license1  car_ownership1  pt_interchanges1   
 
         age_group

problem with fust split not having any node proportion gain ??  - must be equivalent to 0 , think went away 

problem with: - looked at applying rules for both below in except section

- Node Search Split Final Error 

- problem with node purity from counter - this occurs when passing too node search split, with the changed y


splitting with both classes, aka not splitting - i think fixed - but a max on classes applied 

for twoing wth lbt have max_k = 1

 eed to ask about splitting metric with lbt using gini / entropy



#TODO probelm with sending a single string as a class element - dont know if solved this - as for the moment i have the larger list 

In [12]:
tau = False #making y binary 
if tau:
    y = pd.DataFrame(y)
    y["travel_mode"].loc[y["travel_mode"]==2] = 1
    y["travel_mode"].loc[y["travel_mode"]==4] = 3
    y = y["travel_mode"]

setting class as binary for fast test   

In [13]:

my_tree = MyNodeClass('n1', indici) 
cart = CART(y,features,features_names,n_features,n_features_names,impurity_fn = "tau",problem="classifier",method = "TWO-STAGE",twoing = True, max_level = 4, min_cases_parent= 1000,min_cases_child= 500,min_imp_gain=0.0001) 
cart.growing_tree(my_tree, max_k = 1, combination_split = False)


level 0
a [1, 2]
a [1, 3]
a [1, 4]
a [1, 2, 3]
a [1, 2, 4]
a [1, 3, 4]
a [2, 3, 4]
Split Found:  n1 [['distance_group', ('distance03', 'distance02', 'distance01'), 0.3113926939147097, 0]] start
node_proportion_total  0.06902743931139232
level 1
a [1, 2]
a [1, 3]
a [1, 4]
a [1, 2, 3]
a [1, 2, 4]
a [1, 3, 4]
a [2, 3, 4]
Split Found:  n2 [['car_ownership', ('car_ownership2', 'car_ownership1'), 0.2399455853839142, 1]] left
node_proportion_total  0.48261019708335573
Node_proportion_gain  0.4135827577719634
level 2
a [1, 2]
a [1, 3]
a [1, 4]
a [1, 2, 3]
a [1, 2, 4]
a [1, 3, 4]
a [2, 3, 4]
Split Found:  n4 [['distance_group', ('distance04', 'distance05'), 0.1323056233205901, 2]] left
node_proportion_total  0.5253611412796186
Node_proportion_gain  0.04275094419626291
level 3
a [1, 2]
a [1, 3]
a [1, 4]
a [1, 2, 3]
a [1, 2, 4]
a [1, 3, 4]
a [2, 3, 4]
Split Found:  n8 [['distance_group', ('distance06', 'distance04'), 0.052877461853722024, 3]] left
node_proportion_total  0.5501556279335139
Node_pr

(((((None, None), (None, None)), ((None, None), (None, None))),
  (((None, None), (None, None)), ((None, None), None))),
 ((((None, None), (None, None)), ((None, None), None)),
  (((None, None), (None, None)), (None, (None, None)))))

impurity value is the actual impurity in the node accotding to impurity fn aka. gini for the j classes

not for the classes created by twoing the final impuritry measure shown

tomorrow, attempts with cart fast two stage - need to really think about the displayed errors!! worried about tau not decreasing with tree growth , checking if a problem without twoing , no good still may be because of 4 classes 

then regression

then pruning visually


In [15]:

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 3000)
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 


cart.print_tree(table = True, html = True).style.hide_index()

Node,Node_type,Variable_Split,Twoing_Classes_C1,Twoing_Classes_C2,n,Impurity_Value,Class/Value
n1,Parent,"distance_group in ('distance03', 'distance02', 'distance01')","[2, 3, 4]",[1],76103,0.310000,"[{1: 0.18, 2: 0.03, 3: 0.35, 4: 0.44}]"
n2,Parent,"car_ownership in ('car_ownership2', 'car_ownership1')","[1, 2, 3]",[4],58778,0.240000,"[{1: 0.06, 2: 0.03, 3: 0.43, 4: 0.48}]"
n4,Parent,"distance_group in ('distance04', 'distance05')","[2, 3, 4]",[1],16307,0.130000,"[{1: 0.1, 2: 0.05, 3: 0.77, 4: 0.09}]"
n8,Parent,"distance_group in ('distance06', 'distance04')","[2, 3, 4]",[1],12217,0.050000,"[{1: 0.03, 2: 0.05, 3: 0.83, 4: 0.09}]"
n16,Parent,"trafficperc_group in ('0low_traffic', '1mediumlow_traffic')","[1, 2, 3]",[4],10529,0.030000,"[{1: 0.02, 2: 0.04, 3: 0.85, 4: 0.09}]"
n32,Child,None,None,None,7482,None,"[{1: 0.02, 2: 0.05, 3: 0.87, 4: 0.06}]"
n33,Child,None,None,None,3047,None,"[{1: 0.02, 2: 0.03, 3: 0.78, 4: 0.17}]"
n17,Parent,"bus_scale in ('bus_scale0.5', 'bus_scale0.0')","[1, 2]","[3, 4]",1688,0.040000,"[{1: 0.14, 2: 0.06, 3: 0.72, 4: 0.08}]"
n34,Child,None,None,None,951,None,"[{1: 0.18, 2: 0.09, 3: 0.66, 4: 0.07}]"
n35,Child,None,None,None,737,None,"[{1: 0.09, 2: 0.02, 3: 0.8, 4: 0.09}]"
